In [1]:
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv
import os
import openai
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain.callbacks import get_openai_callback
import pandas as pd
from tqdm import tqdm


In [2]:
curated_data = pd.read_csv('../data/benchmark_data/true_false_questions.csv').drop('Unnamed: 0', axis=1)


In [3]:
LLM_MODEL = 'gpt-4'


In [4]:

load_dotenv(os.path.join(os.path.expanduser('~'), '.gpt_config.env'))
API_KEY = os.environ.get('API_KEY')
API_VERSION = os.environ.get('API_VERSION')
RESOURCE_ENDPOINT = os.environ.get('RESOURCE_ENDPOINT')
openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = API_VERSION
chat_deployment_id = LLM_MODEL
chat_model_id = chat_deployment_id
temperature = 0
chat_model = AzureChatOpenAI(openai_api_key=API_KEY, openai_api_version=API_VERSION, azure_deployment=chat_deployment_id, azure_endpoint=RESOURCE_ENDPOINT, temperature=temperature)
load_dotenv(os.path.join(os.path.expanduser('~'), '.spoke_neo4j_config.env'))
username = os.environ.get('NEO4J_USER')
password = os.environ.get('NEO4J_PSW')
url = os.environ.get('NEO4J_URL')
database = os.environ.get('NEO4J_DB')



/Users/karthiksoman/anaconda3/envs/cypher_rag/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [5]:
graph = Neo4jGraph(url=url, username=username, password=password, database = database)
chain = GraphCypherQAChain.from_llm(chat_model, graph=graph, verbose=True, validate_cypher=True,return_intermediate_steps=True)



In [6]:
%%time

cypher_rag_out = []
for index, row in tqdm(curated_data.iterrows()):
    question = 'State if the following Question is True or False and state the reason. \nQuestion: ' + row['text']
    with get_openai_callback() as cb:
        try:
            cypher_rag_answer = chain.run(query=question, return_final_only=True, verbose=False)
            if 'false' in cypher_rag_answer.lower():
                cypher_rag_final_answer = 'False'
            elif 'true' in cypher_rag_answer.lower():
                cypher_rag_final_answer = 'True'
            else:
                cypher_rag_final_answer = None
        except ValueError as e:
            cypher_rag_answer = None
            cypher_rag_final_answer = None
                
    cypher_rag_out.append((row['text'], row['label'], cypher_rag_answer, cypher_rag_final_answer, cb.total_tokens))


0it [00:00, ?it/s]/Users/karthiksoman/anaconda3/envs/cypher_rag/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...


1it [00:02,  2.02s/it]

Generated Cypher:
This task does not require a Cypher statement.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Compound {name: 'metronidazole'})-[:TREATS_CtD]->(d:Disease {name: "crohn's disease"}) RETURN c, d
Full Context:
[]


2it [00:08,  4.87s/it]

...


> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Disease {name: "Doyne honeycomb retinal dystrophy"}) RETURN d.identifier AS Disease_ontology_identifier;
Full Context:
[{'Disease_ontology_identifier': 'DOID:0060745'}]


310it [20:48,  3.71s/it]


> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Disease {name: "Juvenile polyposis syndrome"}), (g:Gene {name: "BMPR1A"}) 
RETURN EXISTS((d)-[:ASSOCIATES_DaG]->(g)) AS result
Full Context:
[]


311it [20:51,  4.03s/it]


> Finished chain.
CPU times: user 15.8 s, sys: 2.02 s, total: 17.9 s
Wall time: 20min 51s


In [9]:
cypher_rag_out_df = pd.DataFrame(cypher_rag_out, columns=['text', 'label', 'cypher_rag_answer', 'cypher_rag_final_answer', 'total_tokens'])


In [15]:

save_path = '../data/results'
os.makedirs(save_path, exist_ok=True)
cypher_rag_out_df.to_csv(os.path.join(save_path, 'cypher_rag_true_false_output.csv'), index=False)

